In [1]:
import findspark

findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("DataFrame ETL - Transformations").getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
spark

In [5]:
sc.defaultParallelism

8

In [6]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

In [7]:
spark.conf.set("park.sql.shuffle.partitions", sc.defaultParallelism)

In [8]:
spark.conf.get("spark.sql.codegen.wholeStage")

'true'

In [9]:
spark.conf.get("spark.submit.deployMode")

'client'

## Preparing Data that will be used in the examples:-
<hr>

In [10]:
from pyspark.sql import functions as F

In [11]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, TimestampType

ord_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("order_date", TimestampType(), False),
    StructField("customer_id", LongType(), False),
    StructField("order_status", StringType(), False)
])

In [12]:
ord_df = spark.read.format("csv").schema( ord_schema ).load("data/orders.csv")

In [13]:
emp_data = [
        ('Robert', 35, 40, 40), ('Robert', 35, 40, 40),
        ('Ram', 31, 33, 29), ('Ram', 31, 33, 91)
]

emp_df = spark.createDataFrame( data = emp_data, schema = ["name", "score1", "score2", "score3"] )

In [14]:
ord_df.show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 15:37:49|      11599|         CLOSED|
|       2|2013-07-25 21:50:25|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
|       6|2013-07-25 00:00:00|       7130|       COMPLETE|
|       7|2013-07-25 00:00:00|       4530|       COMPLETE|
|       8|2013-07-25 00:00:00|       2911|     PROCESSING|
|       9|2013-07-25 00:00:00|       5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSIN

In [15]:
ord_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: timestamp (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- order_status: string (nullable = true)



In [16]:
ord_df.dtypes

[('order_id', 'int'),
 ('order_date', 'timestamp'),
 ('customer_id', 'bigint'),
 ('order_status', 'string')]

In [17]:
emp_df.show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+



In [18]:
emp_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- score1: long (nullable = true)
 |-- score2: long (nullable = true)
 |-- score3: long (nullable = true)



# 1. Selection / Projection:
-----------------------------
- ***select( \*cols )*** : Takes single or multiple column datatype objects as parameter.
- ***selectExpr( \*exprs )*** : Take SQL style expresssions for selection of columns.

In [20]:
ord_df.select( ord_df.order_id, ord_df.order_date ).show(5)

+--------+-------------------+
|order_id|         order_date|
+--------+-------------------+
|       1|2013-07-25 00:00:00|
|       2|2013-07-25 00:00:00|
|       3|2013-07-25 00:00:00|
|       4|2013-07-25 00:00:00|
|       5|2013-07-25 00:00:00|
+--------+-------------------+
only showing top 5 rows



In [22]:
ord_df.select( ord_df.order_status, F.lower(ord_df.order_status).alias("order_status_lowercased") ).show()

+---------------+-----------------------+
|   order_status|order_status_lowercased|
+---------------+-----------------------+
|         CLOSED|                 closed|
|PENDING_PAYMENT|        pending_payment|
|       COMPLETE|               complete|
|         CLOSED|                 closed|
|       COMPLETE|               complete|
|       COMPLETE|               complete|
|       COMPLETE|               complete|
|     PROCESSING|             processing|
|PENDING_PAYMENT|        pending_payment|
|PENDING_PAYMENT|        pending_payment|
| PAYMENT_REVIEW|         payment_review|
|         CLOSED|                 closed|
|PENDING_PAYMENT|        pending_payment|
|     PROCESSING|             processing|
|       COMPLETE|               complete|
|PENDING_PAYMENT|        pending_payment|
|       COMPLETE|               complete|
|         CLOSED|                 closed|
|PENDING_PAYMENT|        pending_payment|
|     PROCESSING|             processing|
+---------------+-----------------

In [35]:
ord_df.selectExpr( "order_date", "date(order_date) AS date_of_order", "substring(order_date, 12, 5) AS time_of_order" ).show()

+-------------------+-------------+-------------+
|         order_date|date_of_order|time_of_order|
+-------------------+-------------+-------------+
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|
|2013-07-25 00:00:00|   2013-07-25|        00:00|


## Unpivoting:-
------------------
- ***stack(n, \*values)*** SQL function, to create a dataframe with ***n*** records, and fitting the whole data in excess of columns.

In [36]:
df_id = spark.range(1)
df_id.show()

+---+
| id|
+---+
|  0|
+---+



In [39]:
df_id.selectExpr( " stack(3, 1,2,3,4,5,6,7,8, 9, 10, 11, 12, 13, 14) " ).show()

+----+----+----+----+----+
|col0|col1|col2|col3|col4|
+----+----+----+----+----+
|   1|   2|   3|   4|   5|
|   6|   7|   8|   9|  10|
|  11|  12|  13|  14|null|
+----+----+----+----+----+



# 2. Creating a new column / Transforming an existing column:-
----------------------------------------------------------------
***withColumn(column_name, \<transformation> )*** - If ***column_name*** matches an existing column name, then the transformation will be applied to that row, else a new column wwill be created with the new name.

In [15]:
ord_df.show(5)

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
+--------+-------------------+-----------+---------------+
only showing top 5 rows



In [19]:
ord_df.withColumn("order_year", F.year(ord_df.order_date) ).show()

+--------+-------------------+-----------+---------------+----------+
|order_id|         order_date|customer_id|   order_status|order_year|
+--------+-------------------+-----------+---------------+----------+
|       1|2013-07-25 00:00:00|      11599|         CLOSED|      2013|
|       2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|      2013|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|      2013|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|      2013|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|      2013|
|       6|2013-07-25 00:00:00|       7130|       COMPLETE|      2013|
|       7|2013-07-25 00:00:00|       4530|       COMPLETE|      2013|
|       8|2013-07-25 00:00:00|       2911|     PROCESSING|      2013|
|       9|2013-07-25 00:00:00|       5657|PENDING_PAYMENT|      2013|
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|      2013|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|      2013|
|      12|2013-07-25

In [21]:
# Giving an alias same as an existing column name, to see that the existing column in the dataframe gets changed.

ord_df.withColumn("order_date", F.year(ord_df.order_date)).show()

+--------+----------+-----------+---------------+
|order_id|order_date|customer_id|   order_status|
+--------+----------+-----------+---------------+
|       1|      2013|      11599|         CLOSED|
|       2|      2013|        256|PENDING_PAYMENT|
|       3|      2013|      12111|       COMPLETE|
|       4|      2013|       8827|         CLOSED|
|       5|      2013|      11318|       COMPLETE|
|       6|      2013|       7130|       COMPLETE|
|       7|      2013|       4530|       COMPLETE|
|       8|      2013|       2911|     PROCESSING|
|       9|      2013|       5657|PENDING_PAYMENT|
|      10|      2013|       5648|PENDING_PAYMENT|
|      11|      2013|        918| PAYMENT_REVIEW|
|      12|      2013|       1837|         CLOSED|
|      13|      2013|       9149|PENDING_PAYMENT|
|      14|      2013|       9842|     PROCESSING|
|      15|      2013|       2568|       COMPLETE|
|      16|      2013|       7276|PENDING_PAYMENT|
|      17|      2013|       2667|       COMPLETE|


In [29]:
ord_df.select( 
    "*",
    F.year(ord_df.order_date).alias("order_year"),
    F.month(ord_df.order_date).alias("order_month"),
    F.dayofmonth(ord_df.order_date).alias("order_day"),
    F.hour(ord_df.order_date).alias("order_hour"),
    F.minute(ord_df.order_date).alias("order_minute"),
    F.second(ord_df.order_date).alias("order_second")
).show()

+--------+-------------------+-----------+---------------+----------+-----------+---------+----------+------------+------------+
|order_id|         order_date|customer_id|   order_status|order_year|order_month|order_day|order_hour|order_minute|order_second|
+--------+-------------------+-----------+---------------+----------+-----------+---------+----------+------------+------------+
|       1|2013-07-25 15:37:49|      11599|         CLOSED|      2013|          7|       25|        15|          37|          49|
|       2|2013-07-25 21:50:25|        256|PENDING_PAYMENT|      2013|          7|       25|        21|          50|          25|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|      2013|          7|       25|         0|           0|           0|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|      2013|          7|       25|         0|           0|           0|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|      2013|          7|       25|      

In [30]:
ord_df.show(5)

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|       1|2013-07-25 15:37:49|      11599|         CLOSED|
|       2|2013-07-25 21:50:25|        256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:00|      12111|       COMPLETE|
|       4|2013-07-25 00:00:00|       8827|         CLOSED|
|       5|2013-07-25 00:00:00|      11318|       COMPLETE|
+--------+-------------------+-----------+---------------+
only showing top 5 rows



### Dropping columns:-
----------------------

In [32]:
ord_df.drop( "order_date", "order_status" ).show()

+--------+-----------+
|order_id|customer_id|
+--------+-----------+
|       1|      11599|
|       2|        256|
|       3|      12111|
|       4|       8827|
|       5|      11318|
|       6|       7130|
|       7|       4530|
|       8|       2911|
|       9|       5657|
|      10|       5648|
|      11|        918|
|      12|       1837|
|      13|       9149|
|      14|       9842|
|      15|       2568|
|      16|       7276|
|      17|       2667|
|      18|       1205|
|      19|       9488|
|      20|       9198|
+--------+-----------+
only showing top 20 rows



### Dropping Duplicate Rows:-
------------------------------

In [33]:
emp_df.show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+



In [35]:
emp_df.dropDuplicates().show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+



In [37]:
emp_df.dropDuplicates(["name", "score1", "score2"]).show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
+------+------+------+------+



# 3. Filtering Data:-
----------------------
1. Below methods/APIs are used to filter data, based on the condition provided. Both the methods works exactly same:
- ***filter()***
- ***where()***

For defining multiple conditions, each condition should be inside () seperately, and they are to be combined using ***&*** , **|** symbols.

2. Also, we can give any SQL type syntax to the above methods, where the whole SQL condition must be inside quotes.

In [38]:
# selecting orders with ids between 10 and 20

ord_df.filter( (ord_df.order_id>=10) & (ord_df.order_id<=20) ).show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSING|
|      15|2013-07-25 00:00:00|       2568|       COMPLETE|
|      16|2013-07-25 00:00:00|       7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:00|       2667|       COMPLETE|
|      18|2013-07-25 00:00:00|       1205|         CLOSED|
|      19|2013-07-25 00:00:00|       9488|PENDING_PAYMENT|
|      20|2013-07-25 00:00:00|       9198|     PROCESSING|
+--------+-------------------+-----------+---------------+



In [39]:
ord_df.where( (ord_df.order_id>=10) & (ord_df.order_id<=20) ).show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSING|
|      15|2013-07-25 00:00:00|       2568|       COMPLETE|
|      16|2013-07-25 00:00:00|       7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:00|       2667|       COMPLETE|
|      18|2013-07-25 00:00:00|       1205|         CLOSED|
|      19|2013-07-25 00:00:00|       9488|PENDING_PAYMENT|
|      20|2013-07-25 00:00:00|       9198|     PROCESSING|
+--------+-------------------+-----------+---------------+



In [41]:
ord_df.where( ord_df.order_id.between(10,20) ).show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSING|
|      15|2013-07-25 00:00:00|       2568|       COMPLETE|
|      16|2013-07-25 00:00:00|       7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:00|       2667|       COMPLETE|
|      18|2013-07-25 00:00:00|       1205|         CLOSED|
|      19|2013-07-25 00:00:00|       9488|PENDING_PAYMENT|
|      20|2013-07-25 00:00:00|       9198|     PROCESSING|
+--------+-------------------+-----------+---------------+



In [42]:
ord_df.where( ord_df.order_id.isin(10,11,12,13,14,15,16,17,18,19,20) ).show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|      10|2013-07-25 00:00:00|       5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:00|       1837|         CLOSED|
|      13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|      14|2013-07-25 00:00:00|       9842|     PROCESSING|
|      15|2013-07-25 00:00:00|       2568|       COMPLETE|
|      16|2013-07-25 00:00:00|       7276|PENDING_PAYMENT|
|      17|2013-07-25 00:00:00|       2667|       COMPLETE|
|      18|2013-07-25 00:00:00|       1205|         CLOSED|
|      19|2013-07-25 00:00:00|       9488|PENDING_PAYMENT|
|      20|2013-07-25 00:00:00|       9198|     PROCESSING|
+--------+-------------------+-----------+---------------+



In [43]:
# selecting will give boolean valued column only

ord_df.select( (ord_df.order_id>=10) & (ord_df.order_id<=20) ).show()

+---------------------------------------+
|((order_id >= 10) AND (order_id <= 20))|
+---------------------------------------+
|                                  false|
|                                  false|
|                                  false|
|                                  false|
|                                  false|
|                                  false|
|                                  false|
|                                  false|
|                                  false|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
|                                   true|
+---------------------------------

### Using SQL syntax:-

In [44]:
ord_df.where( " order_status IN ('CLOSED', 'PENDING') " ).show()

+--------+-------------------+-----------+------------+
|order_id|         order_date|customer_id|order_status|
+--------+-------------------+-----------+------------+
|       1|2013-07-25 15:37:49|      11599|      CLOSED|
|       4|2013-07-25 00:00:00|       8827|      CLOSED|
|      12|2013-07-25 00:00:00|       1837|      CLOSED|
|      18|2013-07-25 00:00:00|       1205|      CLOSED|
|      21|2013-07-25 00:00:00|       2711|     PENDING|
|      24|2013-07-25 00:00:00|      11441|      CLOSED|
|      25|2013-07-25 00:00:00|       9503|      CLOSED|
|      36|2013-07-25 00:00:00|       5649|     PENDING|
|      37|2013-07-25 00:00:00|       5863|      CLOSED|
|      39|2013-07-25 00:00:00|       8214|     PENDING|
|      42|2013-07-25 00:00:00|       9776|     PENDING|
|      44|2013-07-25 00:00:00|      10500|     PENDING|
|      49|2013-07-25 00:00:00|       1871|     PENDING|
|      51|2013-07-25 00:00:00|      12271|      CLOSED|
|      55|2013-07-25 00:00:00|       2052|     P

# 4. Sorting records:-
-----------------------
- ***orderBy( col.asc() OR col.desc() OR col.asc_nulls_first() OR ... )***
- ***sort()***

In [48]:
ord_df.select('order_status').distinct().orderBy( ord_df.order_status.desc() ).show()

+---------------+
|   order_status|
+---------------+
|SUSPECTED_FRAUD|
|     PROCESSING|
|PENDING_PAYMENT|
|        PENDING|
| PAYMENT_REVIEW|
|        ON_HOLD|
|       COMPLETE|
|         CLOSED|
|       CANCELED|
+---------------+



In [53]:
ord_df.select('order_status').distinct().sort( ord_df.order_status.desc() ).show()

+---------------+
|   order_status|
+---------------+
|SUSPECTED_FRAUD|
|     PROCESSING|
|PENDING_PAYMENT|
|        PENDING|
| PAYMENT_REVIEW|
|        ON_HOLD|
|       COMPLETE|
|         CLOSED|
|       CANCELED|
+---------------+



Sorting based on multiple columns, progressively:-

In [52]:
ord_df.orderBy( ord_df.order_status.desc(), ord_df.order_id.asc() ).show()

+--------+-------------------+-----------+---------------+
|order_id|         order_date|customer_id|   order_status|
+--------+-------------------+-----------+---------------+
|      69|2013-07-25 00:00:00|       2821|SUSPECTED_FRAUD|
|     117|2013-07-26 00:00:00|         58|SUSPECTED_FRAUD|
|     246|2013-07-26 00:00:00|       9616|SUSPECTED_FRAUD|
|     320|2013-07-26 00:00:00|      10698|SUSPECTED_FRAUD|
|     329|2013-07-26 00:00:00|       1944|SUSPECTED_FRAUD|
|     411|2013-07-27 00:00:00|      11164|SUSPECTED_FRAUD|
|     423|2013-07-27 00:00:00|       9632|SUSPECTED_FRAUD|
|     453|2013-07-27 00:00:00|        381|SUSPECTED_FRAUD|
|     548|2013-07-28 00:00:00|       6889|SUSPECTED_FRAUD|
|     580|2013-07-28 00:00:00|       8677|SUSPECTED_FRAUD|
|     587|2013-07-28 00:00:00|       6008|SUSPECTED_FRAUD|
|     818|2013-07-29 00:00:00|       8459|SUSPECTED_FRAUD|
|     830|2013-07-29 00:00:00|       9103|SUSPECTED_FRAUD|
|     909|2013-07-30 00:00:00|       8670|SUSPECTED_FRAU

### Sorting the records within its own partition:-
--------------------------------------------------------
- ***sortWithinPartitions()*** - Sorts the records in each partition, and there is no shuffling accross partitions. Extracting the data sorted by this method creates a separate file for each partition, and each file contains sorted records locally (not globally).

In [17]:
test_data = [
    (1,'cap'), (2,'mall'), (3,'jupyter'), (4,'god'), (5, 'ball'), (6, 'zebra'), (7, 'wood'), (8, 'dungeon')
]

test_df = spark.createDataFrame( data = test_data, schema = ["id", "word"] )

In [18]:
test_df.show()

+---+-------+
| id|   word|
+---+-------+
|  1|    cap|
|  2|   mall|
|  3|jupyter|
|  4|    god|
|  5|   ball|
|  6|  zebra|
|  7|   wood|
|  8|dungeon|
+---+-------+



In [23]:
test_df.rdd.getNumPartitions()

8

In [21]:
test_df.rdd.getStorageLevel()

StorageLevel(False, False, False, False, 1)

In [28]:
test_df = test_df.repartition(2)

In [29]:
test_df.rdd.getNumPartitions()

2

In [30]:
test_df.rdd.glom().collect()

[[Row(id=3, word='jupyter'),
  Row(id=4, word='god'),
  Row(id=5, word='ball'),
  Row(id=6, word='zebra'),
  Row(id=7, word='wood')],
 [Row(id=1, word='cap'), Row(id=2, word='mall'), Row(id=8, word='dungeon')]]

In [34]:
test_df = test_df.sortWithinPartitions( test_df.word.asc() )

In [35]:
test_df.rdd.glom().collect()

[[Row(id=5, word='ball'),
  Row(id=4, word='god'),
  Row(id=3, word='jupyter'),
  Row(id=7, word='wood'),
  Row(id=6, word='zebra')],
 [Row(id=1, word='cap'), Row(id=8, word='dungeon'), Row(id=2, word='mall')]]

# 5. Set Opeartions:-
-------------------------
- ***union()*** - Keeps duplicates from spark 2.4+
- ***unionAll()*** - Keeps duplicates
- ***unionByName()*** - Keeps duplicates
- ***intersect()*** - Removes duplicates
- ***intersectAll()*** - Keeps duplicates
- ***exceptAll()*** - Keeps duplicates

In [52]:
set_data1 = [(1,'a'), (1,'a'), (2,'b'), (2,'b'), (3,'c')]
set_data2 = [(1,'a'), (1,'a'), (4,'d'), (5,'e')]

In [53]:
set_df1 = spark.createDataFrame( data = set_data1, schema = ["id", "letter"] )
set_df2 = spark.createDataFrame( data = set_data2, schema = ["id", "letter"] )

In [54]:
set_df1.show()

+---+------+
| id|letter|
+---+------+
|  1|     a|
|  1|     a|
|  2|     b|
|  2|     b|
|  3|     c|
+---+------+



In [42]:
set_df2.show()

+---+------+
| id|letter|
+---+------+
|  1|     a|
|  1|     a|
|  4|     d|
|  5|     e|
+---+------+



In [43]:
set_df1.union( set_df2 ).show()

+---+------+
| id|letter|
+---+------+
|  1|     a|
|  1|     a|
|  2|     b|
|  3|     c|
|  1|     a|
|  1|     a|
|  4|     d|
|  5|     e|
+---+------+



In [44]:
set_df1.unionAll(set_df2).show()

+---+------+
| id|letter|
+---+------+
|  1|     a|
|  1|     a|
|  2|     b|
|  3|     c|
|  1|     a|
|  1|     a|
|  4|     d|
|  5|     e|
+---+------+



In [46]:
set_df1.select("letter", "id").union(set_df2).show()

+------+---+
|letter| id|
+------+---+
|     a|  1|
|     a|  1|
|     b|  2|
|     c|  3|
|     1|  a|
|     1|  a|
|     4|  d|
|     5|  e|
+------+---+



In [47]:
set_df1.select("letter", "id").unionByName(set_df2).show()

+------+---+
|letter| id|
+------+---+
|     a|  1|
|     a|  1|
|     b|  2|
|     c|  3|
|     a|  1|
|     a|  1|
|     d|  4|
|     e|  5|
+------+---+



In [48]:
set_df1.intersect(set_df2).show()

+---+------+
| id|letter|
+---+------+
|  1|     a|
+---+------+



In [49]:
set_df1.intersectAll(set_df2).show()

+---+------+
| id|letter|
+---+------+
|  1|     a|
|  1|     a|
+---+------+



In [55]:
set_df1.exceptAll(set_df2).show()

+---+------+
| id|letter|
+---+------+
|  2|     b|
|  2|     b|
|  3|     c|
+---+------+



# 6. Joins:
-------------
- ***df1.join(df2, df1.col == df2.col, how = inner/outer/left/right/....)***
- ***df1.crossJoin(df2)***

In [17]:
emp_data = [
    (1,'Robert'), (2,'Ria'), (3, 'James')
]

country_data = [(2,'USA'), (4,'India')]

In [18]:
emp_df = spark.createDataFrame( data = emp_data, schema = ["empid", "empname"] )
country_df = spark.createDataFrame( data = country_data, schema = ["empid", "country"] )

In [19]:
emp_df.show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    2|    Ria|
|    3|  James|
+-----+-------+



In [20]:
country_df.show()

+-----+-------+
|empid|country|
+-----+-------+
|    2|    USA|
|    4|  India|
+-----+-------+



### a. Inner Join:-
-----------------------

In [22]:
emp_df.join( country_df, emp_df.empid == country_df.empid, 'inner' ).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    2|    Ria|    2|    USA|
+-----+-------+-----+-------+



### b. Left Outer Join / Left Join:-
--------------------------------------

In [23]:
emp_df.join( country_df, emp_df.empid == country_df.empid, 'left' ).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert| null|   null|
|    2|    Ria|    2|    USA|
|    3|  James| null|   null|
+-----+-------+-----+-------+



### c. Right Outer Join / Right Join:-
----------------------------------------

In [24]:
emp_df.join( country_df, emp_df.empid == country_df.empid, 'right' ).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    2|    Ria|    2|    USA|
| null|   null|    4|  India|
+-----+-------+-----+-------+



### d. Full Outer Join :-
----------------------------------------

In [25]:
emp_df.join( country_df, emp_df.empid == country_df.empid, 'outer' ).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert| null|   null|
|    2|    Ria|    2|    USA|
|    3|  James| null|   null|
| null|   null|    4|  India|
+-----+-------+-----+-------+



### e. Cross Join:-
-----------------------

In [29]:
emp_df.join( country_df, emp_df.empid == emp_df.empid, 'cross' ).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    2|    USA|
|    1| Robert|    4|  India|
|    2|    Ria|    2|    USA|
|    2|    Ria|    4|  India|
|    3|  James|    2|    USA|
|    3|  James|    4|  India|
+-----+-------+-----+-------+



In [27]:
emp_df.crossJoin(country_df).show()

+-----+-------+-----+-------+
|empid|empname|empid|country|
+-----+-------+-----+-------+
|    1| Robert|    2|    USA|
|    1| Robert|    4|  India|
|    2|    Ria|    2|    USA|
|    2|    Ria|    4|  India|
|    3|  James|    2|    USA|
|    3|  James|    4|  India|
+-----+-------+-----+-------+



### f. Left Anti Join:-
-------------------------
Gives the records that are present in the left table, but not in the right table, based on the matching condition.

Basically,
## (Left Anti Join Records) = (Left Join Records) - (Inner Join Records)

Only the columns present in the left table will be shown, as all the right table columns will contan NULL, thus of no use even if present.

In [30]:
emp_df.join( country_df, emp_df.empid == country_df.empid, 'left_anti' ).show()

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    3|  James|
+-----+-------+



### g. Left Semi Join:-
-------------------------

Join type followed is basicaly the inner join. After joining, only the columns of the left table will be in the result set.

In [31]:
emp_df.join( country_df, emp_df.empid == country_df.empid, 'left_semi' ).show()

+-----+-------+
|empid|empname|
+-----+-------+
|    2|    Ria|
+-----+-------+



### h. Self Join :-
--------------------
There is no specific function or API for this join. This a conceptual join, that is accomplished using alias for same dataframe, and then joining them using any of the join types seen above.

- In selft Join, as we use same dataframe as both left and right table, we must set alias for the tables, such that in the join condition, columns of both the dataframes ca be diistinguished.

- As the column names will be accessed by the name of the alias, and not directly by the dataframe name, the column names must be placed as string inside the ***pyspark.sql.functions.col()*** function, else the columns cannot be detected and will throw an error.

In [32]:
emp_data2 = [
    (1,'Robert', 2), (2,'Ria', 3), (3, 'James', 5)
]

emp_df2 = spark.createDataFrame( data = emp_data2, schema = ["empid", "empname", "bossid"] )

In [33]:
emp_df2.show()

+-----+-------+------+
|empid|empname|bossid|
+-----+-------+------+
|    1| Robert|     2|
|    2|    Ria|     3|
|    3|  James|     5|
+-----+-------+------+



#### Inner Self Join:-
-----------------------

In [37]:
emp_df2.alias("t1").join( emp_df2.alias("t2"), F.col("t1.bossid") == F.col("t2.empid"), 'inner' ).show()

+-----+-------+------+-----+-------+------+
|empid|empname|bossid|empid|empname|bossid|
+-----+-------+------+-----+-------+------+
|    1| Robert|     2|    2|    Ria|     3|
|    2|    Ria|     3|    3|  James|     5|
+-----+-------+------+-----+-------+------+



#### Inner Left Join:-
-----------------------

In [38]:
emp_df2.alias("t1").join( emp_df2.alias("t2"), F.col("t1.bossid") == F.col("t2.empid"), "left" ).show()

+-----+-------+------+-----+-------+------+
|empid|empname|bossid|empid|empname|bossid|
+-----+-------+------+-----+-------+------+
|    1| Robert|     2|    2|    Ria|     3|
|    2|    Ria|     3|    3|  James|     5|
|    3|  James|     5| null|   null|  null|
+-----+-------+------+-----+-------+------+



In [39]:
emp_df2.alias("t1").join( emp_df2.alias("t2"), F.col("t1.bossid") == F.col("t2.empid"), "left" ).select( 
    F.col("t1.empid"), 
    F.col("t1.empname"),
    F.col("t1.bossid"),
    F.col("t2.empname").alias("manager_name")
).show()

+-----+-------+------+------------+
|empid|empname|bossid|manager_name|
+-----+-------+------+------------+
|    1| Robert|     2|         Ria|
|    2|    Ria|     3|       James|
|    3|  James|     5|        null|
+-----+-------+------+------------+



### i. Multi Column Join :-
----------------------------

In [19]:
stud_data = [
    (1,'Debanjan', 95), (2,'Projna', 89), (3,'Proman', 77), (4, 'Shaman', 65)
]

marks_data = [
    (100,91,'A+'), (90,81,'A'), (80,71,'B'), (70,61,'C')
]

In [20]:
stud_df = spark.createDataFrame( data = stud_data, schema = ["id", "name", "marks"] )

marks_df = spark.createDataFrame( data = marks_data, schema = ["min_marks", "max_marks", "grade"] )

In [21]:
stud_df.show()

+---+--------+-----+
| id|    name|marks|
+---+--------+-----+
|  1|Debanjan|   95|
|  2|  Projna|   89|
|  3|  Proman|   77|
|  4|  Shaman|   65|
+---+--------+-----+



In [22]:
marks_df.show()

+---------+---------+-----+
|min_marks|max_marks|grade|
+---------+---------+-----+
|      100|       91|   A+|
|       90|       81|    A|
|       80|       71|    B|
|       70|       61|    C|
+---------+---------+-----+



In [23]:
stud_df.join( marks_df, ( ( stud_df.marks >= marks_df.min_marks  ) & ( stud_df.marks <= marks_df.max_marks ) ) , 'left' ).show()

+---+--------+-----+---------+---------+-----+
| id|    name|marks|min_marks|max_marks|grade|
+---+--------+-----+---------+---------+-----+
|  1|Debanjan|   95|     null|     null| null|
|  2|  Projna|   89|     null|     null| null|
|  3|  Proman|   77|     null|     null| null|
|  4|  Shaman|   65|     null|     null| null|
+---+--------+-----+---------+---------+-----+



In [26]:
stud_df.join( marks_df, stud_df.marks.between( marks_df.min_marks, marks_df.max_marks ) , 'left' ).show()

+---+--------+-----+---------+---------+-----+
| id|    name|marks|min_marks|max_marks|grade|
+---+--------+-----+---------+---------+-----+
|  1|Debanjan|   95|     null|     null| null|
|  2|  Projna|   89|     null|     null| null|
|  3|  Proman|   77|     null|     null| null|
|  4|  Shaman|   65|     null|     null| null|
+---+--------+-----+---------+---------+-----+



In [47]:
stud_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- marks: long (nullable = true)



In [48]:
marks_df.printSchema()

root
 |-- min_marks: long (nullable = true)
 |-- max_marks: long (nullable = true)
 |-- grade: string (nullable = true)



In [25]:
stud_df.join(
    marks_df,
    (stud_df["marks"] >= marks_df["min_marks"]) & (stud_df["marks"] <= marks_df["max_marks"]),
    "left"
).show()

+---+--------+-----+---------+---------+-----+
| id|    name|marks|min_marks|max_marks|grade|
+---+--------+-----+---------+---------+-----+
|  1|Debanjan|   95|     null|     null| null|
|  2|  Projna|   89|     null|     null| null|
|  3|  Proman|   77|     null|     null| null|
|  4|  Shaman|   65|     null|     null| null|
+---+--------+-----+---------+---------+-----+



# 7. Aggregation:-
-------------------
- ***summary()*** - Returns a dataframe containing data like count, mean, min, max, standard deviation, percentages, for the dataframe on which this method is applied.
- The rest aggregation functions like ***min(), max(), avg()*** are present in ***pyspark.sql.functions*** module, and they can only be applied inside ***select()*** or ***agg()*** dataframe methods. They take the column name as parameter.

In [30]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, DoubleType

ordItems_schema = StructType([
    StructField("item_id", LongType(), False),
    StructField("order_id", LongType(), False),
    StructField("product_id", LongType(), False),
    StructField("quantity", IntegerType(), False),
    StructField("subtotal", DoubleType(), False),
    StructField("price", DoubleType(), False)
])

In [32]:
ordItems = spark.read.csv("./data/orderItems.csv", schema = ordItems_schema )

In [33]:
ordItems.show()

+-------+--------+----------+--------+--------+------+
|item_id|order_id|product_id|quantity|subtotal| price|
+-------+--------+----------+--------+--------+------+
|      1|       1|       957|       1|  299.98|299.98|
|      2|       2|      1073|       1|  199.99|199.99|
|      3|       2|       502|       5|   250.0|  50.0|
|      4|       2|       403|       1|  129.99|129.99|
|      5|       4|       897|       2|   49.98| 24.99|
|      6|       4|       365|       5|  299.95| 59.99|
|      7|       4|       502|       3|   150.0|  50.0|
|      8|       4|      1014|       4|  199.92| 49.98|
|      9|       5|       957|       1|  299.98|299.98|
|     10|       5|       365|       5|  299.95| 59.99|
|     11|       5|      1014|       2|   99.96| 49.98|
|     12|       5|       957|       1|  299.98|299.98|
|     13|       5|       403|       1|  129.99|129.99|
|     14|       7|      1073|       1|  199.99|199.99|
|     15|       7|       957|       1|  299.98|299.98|
|     16| 

In [29]:
ordItems.count()

172198

In [34]:
ordItems.printSchema()

root
 |-- item_id: long (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- subtotal: double (nullable = true)
 |-- price: double (nullable = true)



In [35]:
ordItems.summary().show()

+-------+-----------------+------------------+-----------------+------------------+------------------+-----------------+
|summary|          item_id|          order_id|       product_id|          quantity|          subtotal|            price|
+-------+-----------------+------------------+-----------------+------------------+------------------+-----------------+
|  count|           172198|            172198|           172198|            172198|            172198|           172198|
|   mean|          86099.5| 34442.56682423721|660.4877176273824|2.1821275508426345|199.32066533874877|133.7590662494616|
| stddev|49709.42516431533|19883.325171992343| 310.514472790008|1.4663523175387134|112.74303721400686|118.5589325726674|
|    min|                1|                 1|               19|                 1|              9.99|             9.99|
|    25%|            43033|             17204|              403|                 1|            119.98|             50.0|
|    50%|            86085|     

In [36]:
# Applying summary() method on a subset of columns

ordItems.select( 'quantity', 'price' ).summary().show()

+-------+------------------+-----------------+
|summary|          quantity|            price|
+-------+------------------+-----------------+
|  count|            172198|           172198|
|   mean|2.1821275508426345|133.7590662494616|
| stddev|1.4663523175387134|118.5589325726674|
|    min|                 1|             9.99|
|    25%|                 1|             50.0|
|    50%|                 1|            59.99|
|    75%|                 3|           199.99|
|    max|                 5|          1999.99|
+-------+------------------+-----------------+



In [46]:
# Find average price and average subtotal of all the orders:-

ordItems.groupBy("order_id").agg( 
    F.avg("price"), 
    F.round(F.avg("subtotal"), 2), 
    F.min("subtotal"), 
    F.max("subtotal") 
).orderBy( ordItems.order_id.asc() ).show()

+--------+------------------+-----------------------+-------------+-------------+
|order_id|        avg(price)|round(avg(subtotal), 2)|min(subtotal)|max(subtotal)|
+--------+------------------+-----------------------+-------------+-------------+
|       1|            299.98|                 299.98|       299.98|       299.98|
|       2|126.66000000000001|                 193.33|       129.99|        250.0|
|       4|             46.24|                 174.96|        49.98|       299.95|
|       5|           167.984|                 225.97|        99.96|       299.98|
|       7|171.98666666666668|                 193.31|        79.95|       299.98|
|       8|             54.99|                 182.46|         50.0|       299.95|
|       9|166.65666666666667|                 199.99|       199.98|       199.99|
|      10|           120.388|                 130.38|        21.99|       199.99|
|      11|59.986000000000004|                 183.96|        49.98|       399.96|
|      12|104.99

In [41]:
ordItems.agg( F.min("subtotal"), F.max("subtotal"), F.min("price"), F.max("price") ).show()

+-------------+-------------+----------+----------+
|min(subtotal)|max(subtotal)|min(price)|max(price)|
+-------------+-------------+----------+----------+
|         9.99|      1999.99|      9.99|   1999.99|
+-------------+-------------+----------+----------+



In [47]:
ordItems.select( F.round(F.avg("price"), 2), F.min("price"), F.max("price")  ).show()

+--------------------+----------+----------+
|round(avg(price), 2)|min(price)|max(price)|
+--------------------+----------+----------+
|              133.76|      9.99|   1999.99|
+--------------------+----------+----------+



- ***sum()***
- ***sumDistinct()*** - deprecated, and the newer version of this function is ***sum_distinct()***

In [51]:
test_df = spark.createDataFrame( data = [(1,),(2,),(3,),(3,),(4,),(5,)], schema = ["value"] )
test_df.show()

+-----+
|value|
+-----+
|    1|
|    2|
|    3|
|    3|
|    4|
|    5|
+-----+



In [52]:
test_df.select( F.sum("value"), F.sumDistinct("value") ).show()

D:\Softwares\Apache_Spark\spark\python\pyspark\sql\functions.py:316: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------+-------------------+
|sum(value)|sum(DISTINCT value)|
+----------+-------------------+
|        18|                 15|
+----------+-------------------+



In [53]:
test_df.select( F.sum("value"), F.sum_distinct("value") ).show()

+----------+-------------------+
|sum(value)|sum(DISTINCT value)|
+----------+-------------------+
|        18|                 15|
+----------+-------------------+



- ***count()***
- ***countDistinct()***

In [54]:
test_df.select( F.count("value"), F.countDistinct("value") ).show()

+------------+---------------------+
|count(value)|count(DISTINCT value)|
+------------+---------------------+
|           6|                    5|
+------------+---------------------+



In [56]:
test_df.agg( 
    F.count("value").alias("count"), 
    F.count_distinct("value").alias("count_distinct"), 
    F.countDistinct("value").alias("countDistinct") 
).show()

+-----+--------------+-------------+
|count|count_distinct|countDistinct|
+-----+--------------+-------------+
|    6|             5|            5|
+-----+--------------+-------------+



### Aggregating all values into a collection:-
------------------------------------------------
- ***collect_set()*** - Aggregates all the values in a group(if ***groupBy()*** is applied), or all values in a column, into a Python set (thus, does not contains duplicates).
- ***collect_list()*** - Aggregates all the values in a group(if ***groupBy()*** is applied), or all values in a column, into a Python list (contains duplicates).

As in the result set the attributes actually contaiin Python collections, thus they can be indexed also. But, Slicing dicing operation is ***not applicable***.

In [59]:
test_df.select( F.collect_set("value"), F.collect_list("value") ).show()

+------------------+-------------------+
|collect_set(value)|collect_list(value)|
+------------------+-------------------+
|   [1, 5, 2, 3, 4]| [1, 2, 3, 3, 4, 5]|
+------------------+-------------------+



In [60]:
test_df.select( F.collect_set("value")[0], F.collect_list("value")[0] ).show()

+---------------------+----------------------+
|collect_set(value)[0]|collect_list(value)[0]|
+---------------------+----------------------+
|                    1|                     1|
+---------------------+----------------------+



In [64]:
test_df.select( F.collect_set("value")[0:3], F.collect_list("value")[0:3] ).show()

AnalysisException: cannot resolve 'substring(collect_set(value), 0, 3)' due to data type mismatch: argument 1 requires (string or binary) type, however, 'collect_set(value)' is of array<bigint> type.;
'Aggregate [unresolvedalias(substring(collect_set(value#1608L, 0, 0), 0, 3), Some(org.apache.spark.sql.Column$$Lambda$3915/0x0000000101651040@60fa347e)), unresolvedalias(substring(collect_list(value#1608L, 0, 0), 0, 3), Some(org.apache.spark.sql.Column$$Lambda$3915/0x0000000101651040@60fa347e))]
+- LogicalRDD [value#1608L], false


### Functions related to statistical analysis:
------------------------------------------------
- ***skewness()***
- ***variance()***
- ***stddev()*** - Standard Deviation

In [65]:
test_df.select( F.skewness("value"), F.variance("value"), F.stddev("value") ).show()

+---------------+------------------+------------------+
|skewness(value)|   var_samp(value)|stddev_samp(value)|
+---------------+------------------+------------------+
|            0.0|1.9999999999999996| 1.414213562373095|
+---------------+------------------+------------------+



# 8. Grouping and Pivot:-
--------------------------

In [66]:
emp_data = [
    ("James", "Sales", "NY", 9000,34), 
    ("Alicia", "Sales", "NY", 8600,56), 
    ("Robert", "Sales", "CA", 8100, 30), 
    ("Lisa", "Finance", "CA", 9000, 24), 
    ("Deja", "Finance", "CA", 9900, 40), 
    ("Sugie", "Finance", "NY", 8300, 36), 
    ("Ram", "Finance", "NY", 7900,53), 
    ("Kyle", "Marketing", "CA", 8000, 25), 
    ("Reid", "Marketing", "NY", 9100, 50)
]

emp_schema = ["empname", "dept", "state", "salary", "age"]

emp_df = spark.createDataFrame( data = emp_data, schema = emp_schema )

In [67]:
emp_df.show()

+-------+---------+-----+------+---+
|empname|     dept|state|salary|age|
+-------+---------+-----+------+---+
|  James|    Sales|   NY|  9000| 34|
| Alicia|    Sales|   NY|  8600| 56|
| Robert|    Sales|   CA|  8100| 30|
|   Lisa|  Finance|   CA|  9000| 24|
|   Deja|  Finance|   CA|  9900| 40|
|  Sugie|  Finance|   NY|  8300| 36|
|    Ram|  Finance|   NY|  7900| 53|
|   Kyle|Marketing|   CA|  8000| 25|
|   Reid|Marketing|   NY|  9100| 50|
+-------+---------+-----+------+---+



In [69]:
help( emp_df.groupBy("dept") )

Help on GroupedData in module pyspark.sql.group object:

class GroupedData(pyspark.sql.pandas.group_ops.PandasGroupedOpsMixin)
 |  GroupedData(jgd: py4j.java_gateway.JavaObject, df: pyspark.sql.dataframe.DataFrame)
 |  
 |  A set of methods for aggregations on a :class:`DataFrame`,
 |  created by :func:`DataFrame.groupBy`.
 |  
 |  .. versionadded:: 1.3
 |  
 |  Method resolution order:
 |      GroupedData
 |      pyspark.sql.pandas.group_ops.PandasGroupedOpsMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, jgd: py4j.java_gateway.JavaObject, df: pyspark.sql.dataframe.DataFrame)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  agg(self, *exprs: Union[pyspark.sql.column.Column, Dict[str, str]]) -> pyspark.sql.dataframe.DataFrame
 |      Compute aggregates and returns the result as a :class:`DataFrame`.
 |      
 |      The available aggregate functions can be:
 |      
 |      1. built-in aggregation functions, such as `av

In [75]:
emp_df.groupBy("dept").agg( 
    F.sum("salary"),
    F.avg("salary"),
    F.min("salary"),
    F.max("salary"),
    F.count("salary")
).show()

+---------+-----------+-----------------+-----------+-----------+-------------+
|     dept|sum(salary)|      avg(salary)|min(salary)|max(salary)|count(salary)|
+---------+-----------+-----------------+-----------+-----------+-------------+
|    Sales|      25700|8566.666666666666|       8100|       9000|            3|
|  Finance|      35100|           8775.0|       7900|       9900|            4|
|Marketing|      17100|           8550.0|       8000|       9100|            2|
+---------+-----------+-----------------+-----------+-----------+-------------+



### Grouping multiple columns:-
---------------------------------

In [76]:
emp_df.groupBy( emp_df.dept, emp_df.state ).min( "salary", "age" ).show()

+---------+-----+-----------+--------+
|     dept|state|min(salary)|min(age)|
+---------+-----+-----------+--------+
|    Sales|   NY|       8600|      34|
|    Sales|   CA|       8100|      30|
|  Finance|   CA|       9000|      24|
|  Finance|   NY|       7900|      36|
|Marketing|   NY|       9100|      50|
|Marketing|   CA|       8000|      25|
+---------+-----+-----------+--------+



### Applying filter before and after grouping:-
-------------------------------------------------
No separate clause for ***HAVING***. Instead, applying ***where()*** or ***filter()*** after grouping does the work.

In [78]:
emp_df.where( emp_df.state=='NY' ).groupBy('dept').agg( F.sum("salary") ).show()

+---------+-----------+
|     dept|sum(salary)|
+---------+-----------+
|    Sales|      17600|
|  Finance|      16200|
|Marketing|       9100|
+---------+-----------+



In [79]:
emp_df.where( emp_df.state=='NY' ).groupBy('dept').agg( 
    F.sum("salary").alias("total_salary") 
).where( F.col('total_salary') > 10000 ).show()

+-------+------------+
|   dept|total_salary|
+-------+------------+
|  Sales|       17600|
|Finance|       16200|
+-------+------------+



## Pivot:-
-------------

- ***df.groupBy(\<group_column_name>).pivot(\<pivot_column_name>).\<aggregate_function>(\<aggregate_column_name>)***

In [81]:
sub_df = emp_df.select("dept", "state", "salary")
sub_df.show()

+---------+-----+------+
|     dept|state|salary|
+---------+-----+------+
|    Sales|   NY|  9000|
|    Sales|   NY|  8600|
|    Sales|   CA|  8100|
|  Finance|   CA|  9000|
|  Finance|   CA|  9900|
|  Finance|   NY|  8300|
|  Finance|   NY|  7900|
|Marketing|   CA|  8000|
|Marketing|   NY|  9100|
+---------+-----+------+



In [82]:
sub_df.groupBy( "dept", "state" ).sum("salary").show()

+---------+-----+-----------+
|     dept|state|sum(salary)|
+---------+-----+-----------+
|    Sales|   NY|      17600|
|    Sales|   CA|       8100|
|  Finance|   CA|      18900|
|  Finance|   NY|      16200|
|Marketing|   NY|       9100|
|Marketing|   CA|       8000|
+---------+-----+-----------+



- For pivot, of the two (or multiple) group columns, we just take out and make one column as the **Pivot Column**, whose values are to be converted as the column names, and the rest aggregation will be same.

- Pivot column will be defined by passing the column name as the parameter of the ***pivot()*** method. This method is to be used **after grouping** and **before aggregation** in pyspark.

- When using pivot, only one aggregation of a column is allowed.

In [83]:
sub_df.groupBy( sub_df.dept ).pivot( "state" ).sum("salary").show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



In [92]:
sub_df.groupBy( sub_df.state ).pivot( "dept" ).sum("salary").show()

+-----+-------+---------+-----+
|state|Finance|Marketing|Sales|
+-----+-------+---------+-----+
|   CA|  18900|     8000| 8100|
|   NY|  16200|     9100|17600|
+-----+-------+---------+-----+



In [84]:
sub_df.groupBy( sub_df.dept ).pivot( "state" ).agg( F.sum("salary")).show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



### Using Pivot with a single group column:-
----------------------------------------------

In [91]:
emp_df.select("dept", "salary").groupBy("dept").pivot("dept").sum("salary").show()

+---------+-------+---------+-----+
|     dept|Finance|Marketing|Sales|
+---------+-------+---------+-----+
|    Sales|   null|     null|25700|
|  Finance|  35100|     null| null|
|Marketing|   null|    17100| null|
+---------+-------+---------+-----+



# 9. Unpivot:-
----------------
No specific function exists for unpivoting. It is done using ***spark.sql()*** of ***df.selectExpr()***, as use of ***stack()*** function is necessary.

In [95]:
dummy_df = spark.createDataFrame( data = [('a',),('b',)], schema = ["id"] )
dummy_df.show()

+---+
| id|
+---+
|  a|
|  b|
+---+



In [97]:
# First parameter of stack() fuction defines the number of rows would be there.

dummy_df.selectExpr( "stack(3, 1,2,3,4,5,6,7,8,9,0,10,11,12,13,14,15,16)" ).show()

+----+----+----+----+----+----+
|col0|col1|col2|col3|col4|col5|
+----+----+----+----+----+----+
|   1|   2|   3|   4|   5|   6|
|   7|   8|   9|   0|  10|  11|
|  12|  13|  14|  15|  16|null|
|   1|   2|   3|   4|   5|   6|
|   7|   8|   9|   0|  10|  11|
|  12|  13|  14|  15|  16|null|
+----+----+----+----+----+----+



In [98]:
dummy_df.selectExpr( "id", "stack(3, 1,2,3,4,5,6,7,8,9,0,10,11,12,13,14,15,16)" ).show()

+---+----+----+----+----+----+----+
| id|col0|col1|col2|col3|col4|col5|
+---+----+----+----+----+----+----+
|  a|   1|   2|   3|   4|   5|   6|
|  a|   7|   8|   9|   0|  10|  11|
|  a|  12|  13|  14|  15|  16|null|
|  b|   1|   2|   3|   4|   5|   6|
|  b|   7|   8|   9|   0|  10|  11|
|  b|  12|  13|  14|  15|  16|null|
+---+----+----+----+----+----+----+



In [99]:
pivot_df = sub_df.groupBy("dept").pivot("state").sum("salary")
pivot_df.show()

+---------+-----+-----+
|     dept|   CA|   NY|
+---------+-----+-----+
|    Sales| 8100|17600|
|  Finance|18900|16200|
|Marketing| 8000| 9100|
+---------+-----+-----+



In [103]:
pivot_df.selectExpr("dept", " stack(2, 'CA', CA, 'NY', NY) as (state,salary) ").show()

+---------+-----+------+
|     dept|state|salary|
+---------+-----+------+
|    Sales|   CA|  8100|
|    Sales|   NY| 17600|
|  Finance|   CA| 18900|
|  Finance|   NY| 16200|
|Marketing|   CA|  8000|
|Marketing|   NY|  9100|
+---------+-----+------+

